import collections
import os
import random
import tarfile
import torch
from torch import nn
import torchtext.vocab as Vocab
import torch.utils.data as Data
import pandas as pd
import numpy as np
import time
import torch.nn.functional as F
from numpy import savetxt
import sys

sys.path.append("..")
device=torch.device('cuda' if torch.cuda.is_available else 'cpu')

In [9]:
import collections
import os
import random
import tarfile
import torch
from torch import nn
import torchtext.vocab as Vocab
import torch.utils.data as Data
from torch.utils.data import dataloader, Dataset, TensorDataset
import pandas as pd
import numpy as np
import time
import torch.nn.functional as F
from numpy import savetxt

import sys
sys.path.append("..")
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [2]:
df_train = pd.read_csv('D:\计算机\代码\FudanNLP_Begginer-main\FudanNLP_Begginer-main\data/train.tsv',header = 0, delimiter = '\t')
df_test = pd.read_csv('D:\计算机\代码\FudanNLP_Begginer-main\FudanNLP_Begginer-main\data/test.tsv',header = 0, delimiter = '\t')
df_train.info()  #对pandas读取的数据给出一个summary

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156060 entries, 0 to 156059
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   PhraseId    156060 non-null  int64 
 1   SentenceId  156060 non-null  int64 
 2   Phrase      156060 non-null  object
 3   Sentiment   156060 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 4.8+ MB


In [3]:
df_train.head()  #返回读取数据的前五行

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [4]:
df_train.Sentiment.value_counts()/df_train.Sentiment.count()

2    0.509945
3    0.210989
1    0.174760
4    0.058990
0    0.045316
Name: Sentiment, dtype: float64

In [5]:
df_train.Phrase.str.len().max()   #找数据中最长的一个

283

In [6]:
df_test.head()

,PhraseId,SentenceId,Phrase
0,156061,8545,An intermittently pleasing but mostly routine ...
1,156062,8545,An intermittently pleasing but mostly routine ...
2,156063,8545,An
3,156064,8545,intermittently pleasing but mostly routine effort
4,156065,8545,intermittently pleasing but mostly routine


In [1]:
X = df_train['Phrase']
Y = df_train['Sentiment']
ceshi_data=df_test['Phrase']
all_data=list(X) #转化为list
all_labels=list(Y)
c=list(zip(all_data,all_labels)) #data和label一对一压起来
random.shuffle(c)
all_data[:],all_labels[:]=zip(*c)#解压
length_train=int(len(all_data)*0.8)
train_data=all_data[:length_train]
train_labels=all_labels[:length_train]
test_data=all_data[length_train:]
test_labels=all_labels[length_train:]
len(train_data),len(train_labels),len(test_data),len(test_labels)

NameError: name 'df_train' is not defined

In [8]:
def get_tokenized_sentiment(data):
    def tokenizer(text):
        return[tok.lower() for tok in text.split(' ')]   #转换大小写，分割text
    return[tokenizer(review) for review in data]

In [21]:
def get_vocab_sentiment(data):
    tokenized_data=get_tokenized_sentiment(data)
    counter=collections.Counter([tk for st in tokenized_data for tk in st])

    return Vocab.vocab(counter)   #排除出现五次以下的sentiment，返回一个字典    删除 min_freq=5
vocab=get_vocab_sentiment(all_data)
'#words in vocab:',len(vocab)

('#words in vocab:', 16532)

In [22]:
def preprocess_sentiment(data,vocab,labels=None):
    max_l=250
    def pad(x):
        return x[:max_l] if len(x)>max_l else x+[0]*(max_l-len(x))
    tokenized_data=get_tokenized_sentiment(data)
    features=torch.tensor([pad([vocab.__getitem__(word)for word in words])for words in tokenized_data])  #stoi()–>将字符串转换为int, 然后pad每一条评论
    if labels:
        labels=torch.tensor(labels)    #????????????
        return features, labels
    else:
        return features

In [40]:
batch_size=256
train_set=TensorDataset(*preprocess_sentiment(train_data,vocab,train_labels))
test_set=TensorDataset(*preprocess_sentiment(test_data,vocab,test_labels))
train_iteration=dataloader(train_set,batch_size,shuffle=True)
test_iteration=dataloader(test_set,batch_size)

TypeError: 'module' object is not callable

In [10]:
class BiRNN(nn.Module):
    
    def __init__(self,vocab,embed_size,num_hidden,num_layers):
        super(BiRNN,self).__init__()
        self.embedding=nn.Embedding(len(vocab),embed_size)
        self.encoder=nn.LSTM(input_size=embed_size,hidden_size=num_hidden,num_layers=num_layers,bidirectional=True)
        #双向，最终步和最初步，所以是4*num_hiddens
        self.decoder=nn.Linear(4*num_hidden,5)  #输入维度4*num，输出维度为5，训练一个线性函数来拟合
    def forward(self,inputs):
        embedding=self.embedding(inputs.permute(1,0)) #permute 交换tensor维度
        outputs,_=self.encoder(embedding)#输出(h,c)
        encoding=torch.cat(outputs(0),output(-1),-1) #按最后一维拼接
        out=self.decoder(encoding)
        return outs
    
    
    
    

In [11]:
embed_size,hidden_nums,hidden_layers=100,100,2
net_withpretrained=BiRNN(vocab,embed_size,num_hidden,hum_layers)
net.withoutpretrained=BiRNN(vocab,embed_size,num_hidden,hum_layers)

NameError: name 'vocab' is not defined

In [12]:
glove_vocab=Vocab.GloVe(name='6B',dim=100,cache='D:\计算机\代码\FudanNLP_Begginer-main\FudanNLP_Begginer-main/data/glove')

D:\计算机\代码\FudanNLP_Begginer-main\FudanNLP_Begginer-main/data/glove\glove.6B.zip: 862MB [09:49, 1.46MB/s]          
100%|██████████████████████████████████████████████████████████████████████▉| 399999/400000 [00:19<00:00, 20237.13it/s]


In [28]:
def load_pretrain_embedding(words,pretrain_vocab):
    embed=torch.zeros(len(words),pretrain_vocab.vectors[0].shape[0])
    oov_count=0
    for i,word in enumerate(words):   #enumerate() 函数用于将一个可遍历的数据对象(如列表、元组或字符串)组合为一个索引序列
        
        #异常检测   
        try:
            idx=pretrain_vocab.get_stoi(word) #stoi   现版本用法
            embed[i,:]=pretrain_vocab.vectors[idx]
        except KeyError:  #KeyError 映射中没有这个键，此处为无对应的词向量
            oov_count+=1
        if oov_count>0:
            print("There is %d OOV words" %oov_count)
            return embed
  

In [ ]:
net_withpretrained.embedding.weight.data.copy_(vocab.itos,glove_vocab)#tensor.copy_ 从vocab复制数据
# nn.Embedding具有一个权重（.weight），形状是(num_words, embedding_dim)
net_withpretrained.embedding.weight.requires_grad=False
#requires_grad=True 则为这个tensor计算梯度    已经是预训练好的词向量，这里就不再需要进行更新

In [ ]:
def evaluate_accuracy(data_iter,net,device=None):
    if device is None and isinstsance(net,torch.nn.Module):    #isinstance() 函数来判断一个对象是否是一个已知的类型
        device=list(net.parameter())[0].device
        acc_sum,n=0.0,0
    with torch.no_grad():
        for X,y in data_iter:
            if isinstance(net,nn.Module):
                net.eval()# 评估模式, 这会关闭dropout
                acc_sum += (net(X.to(device)).argmax(dim=1) == y.to(device)).float().sum().cpu().item()
                net.train()# 改回训练模式
            else:
                if('is_training' in net.__code__.co_varnames): # 如果有is_training这个参数
                    # 将is_training设置成False
                    # items()的作用是把字典中的每对key和value组成一个元组，并把这些元祖放在列表中返回，item()函数取出的元素值的精度更高，所以在求损失函数等时我们一般用item（）
                    #argmax返回指定维度最大值的序号
                    acc_sum += (net(X, is_training=False).argmax(dim=1) == y).float().sum().item() #acc sum了什么
                else:
                    acc_sum += (net(X).argmax(dim=1) == y).float().sum().item() 
            n+=y.shape[0]
        return acc_sum/n
    
    

In [30]:
def train(train_iter,test_iter,net, loss,optimizer,device,num_epochs):
    #model.to(cuda)   copy model to cuda (or cpu) 将所有最开始读取数据时的tensor变量copy一份到device所指定的GPU上去，之后的运算都在GPU上进行
    #equveliant to model.cuda()
    #torch.device代表将torch.Tensor分配到的设备的对象。
    #torch.device包含一个设备类型（'cpu'或'cuda'设备类型）和可选的设备的序号。
        net=net.to(device)
        print('training on',device)
        batch_count=0
        for epoch in range(num_epochs):
            train_1_sum,train_acc_sum,n,start=0.0,0.0,0,time.time()
            for X,y in train_iter:
                X=X.to(device)
                y=y.to(device)
                y_hat=net(X)
                l=loss(y_hat,y)
                #清零梯度，反向传播，计算梯度
                optimizer.zero_grad()
                l.backward()
                optimizer.step()
                train_1_sum+=cpu().sum().item() #sum loss of each batch
                # argmax返回矩阵y_hat每行中最大元素的索引，且返回结果与变量y形状相同
                # 相等条件判断式(y_hat.argmax(dim=1) == y)是一个类型为ByteTensor的Tensor
                # 判断式结果为值为0（相等为假）或1（相等为真）的Tensor，然后sum求和，得到判断正确的次数，再求平均得到正确率
                train_acc_sum=+=(y_hat.argmax(dim=1)==y).sum().cpu().item()
                n+=yshape(0)
                batch_count+=1
        test_acc=evaluate_accuracy(test_iter,net)
        print('epochs %d: loss %4f: train accuracy %3f: test accuracy %3f: time %f: sec'%
               epochs+1,train_1_sum,train_acc_sum/n,test_acc,time.time()-start)
    
    

In [32]:
lr,num_epochs=0.01,5
#lambda arg : expression，解释：冒号前是参数，冒号后是返回值。
#filter根据判断结果自动过滤掉不符合条件的元素，并返回有符合元素组成的新列表
#filter(lambda p: p.requires_grad, model.parameters())中 lambda p: p.requires_grad就是以p为参数的满足p.requires_grad的true的条件的函数。
#而参数p赋值的元素从列表model.parameters()里取。所以只取param.requires_grad = True（模型参数的可导性是true的元素），就过滤掉为false的元素。
optimizer=torch.optim.Adam(filter(lambda p:p.requires_grad,net_withpretrained.parameters()),lr=lr)
loss=bb.CrossEntropyLoss()
train(train_iter,test_iter,net,loss,optimier,devicenum_epochs)

NameError: name 'net_withpretrained' is not defined

In [ ]:
lr,num_epochs=0.01,5
optimizer=torch.optim.Adam(net.parameters(),lr=lr)
loss=nn.CrossEntropyLoss()
train(train_iter,test_iter,net_witoutpretrained,loss,optimizer,device,num_epochs)

In [33]:
def test_predict(data_iter,net,device):
    net=net.to(device)
    print('training on',device)
    predict=[]
    i=0
    start=time.time()
    for X in data_iter:
        X=X.to(device)
        y=net(X.view(1,-1).argmax(dim=1))
        predict.append(y.cpu().item())
        i+=1
        if (i%5000==0):
            print('%d samples predicted,%d samples left, cost time %f'% (i,(ceshi_num-i),time.time()-star))
            start=time.time()
        return predict
ceshi_iter=iter(ceshi_set)
predict_list=test_predict(ceshi_iter,net_withpretrained,device)
                  
    
    

NameError: name 'ceshi_set' is not defined

In [35]:
pred=[[index+156061,x]for index,x in enumerate(predict_list)]
savetxt('D:\计算机\代码\FudanNLP_Begginer-main\FudanNLP_Begginer-main\sentiment-analysis-on-movie-reviews/rnn_benchmark.csv',pred,dilimiter=',',fmt='%d,%d',header='PhraseID,Sentiment',comment='')

NameError: name 'predict_list' is not defined

In [26]:
a=torch.randn([1,3]) 
b=0 
for i in range(3): 
    b+=(torch.randn([1,3]).argmax(dim=0)==a[i]).float().sum().item() 
    print(b)

0.0


IndexError: index 1 is out of bounds for dimension 0 with size 1